In [ ]:
import dlib # conda install -c conda-forge cmake dlib
import cv2 # pip install opencv-python
import numpy as np # pip install numpy

# dlib의 얼굴 감지기와 랜드마크 예측기 로드
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # 랜드마크 모델 다운로드 필요

# 눈 거리 비율(EAR) 계산 함수
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # 왼쪽 눈과 오른쪽 눈 좌표 가져오기
        left_eye = np.array([[landmarks.part(n).x, landmarks.part(n).y] for n in range(36, 42)])
        right_eye = np.array([[landmarks.part(n).x, landmarks.part(n).y] for n in range(42, 48)])

        # EAR 계산
        EAR_left = eye_aspect_ratio(left_eye)
        EAR_right = eye_aspect_ratio(right_eye)
        EAR_avg = (EAR_left + EAR_right) / 2.0

        # EAR 값이 0.2 이하이면 눈 감김으로 판단
        if EAR_avg < 0.2:
            cv2.putText(frame, "Eyes Closed!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Eyes Open", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 눈 부분 표시
        for point in left_eye:
            cv2.circle(frame, tuple(point), 2, (255, 0, 0), -1)
        for point in right_eye:
            cv2.circle(frame, tuple(point), 2, (255, 0, 0), -1)

    cv2.imshow("Eye Blink Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()